<a href="https://colab.research.google.com/github/Lufec/Caso_Dostoy/blob/main/Dostoy_2classes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split

In [ ]:
np.set_printoptions(precision = 5,edgeitems = 11, linewidth = 200, suppress = True)

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
df = pd.read_excel("/content/gdrive/My Drive/CIÊNCIA DA COMPUTAÇÃO/CNN/Dolstoy_Profile.xls").drop(columns = 'ClienteNo')

Separar target em labels

In [ ]:
df.PrimeiraCompra.describe()

count    2840.000000
mean       53.112063
std        26.237098
min         0.000000
25%        32.900000
50%        51.010000
75%        71.360000
max       137.780000
Name: PrimeiraCompra, dtype: float64

In [ ]:
df['Lucro'] = df['PrimeiraCompra'].copy() - 42

In [ ]:
df.NivelGerencial = df.NivelGerencial -1

In [ ]:
df.loc[df.PrimeiraCompra < 42, "PrimeiraCompra"] = 0
df.loc[df.PrimeiraCompra >= 42, "PrimeiraCompra"] = 1

In [ ]:
df.PrimeiraCompra.value_counts()

1.0    1776
0.0    1064
Name: PrimeiraCompra, dtype: int64

In [ ]:
df_caso1 = df[df['PrimeiraCompra'] == 1]
df_caso0 = df[df['PrimeiraCompra'] == 0]

In [ ]:
df_caso1['PrimeiraCompra'].value_counts()

1.0    1776
Name: PrimeiraCompra, dtype: int64

Separar treino balanceadamente (851 casos "1" e 851 casos "0"). Teste fica com o resto (925 casos "1" e 213 casos "0")

In [ ]:
df1_train = df_caso1.head(851)
df0_train = df_caso0.head(851)
df1_test  = df_caso1.tail(925)
df0_test  = df_caso0.tail(213)
df_train = pd.concat([df1_train,df0_train]).sample(frac = 1).reset_index(drop = True)
df_test = pd.concat([df1_test,df0_test]).sample(frac = 1).reset_index(drop = True)
x_train = df_train.drop(columns = 'PrimeiraCompra')
y_train = df_train.PrimeiraCompra
x_test = df_test.drop(columns = 'PrimeiraCompra')
y_test = df_test.PrimeiraCompra

In [ ]:
x_train.head(5)

,Cidade,Idade,Sexo,EstadoCivil,NumeroFilhos,Escolaridade,Ocupacao,AreaAtuacao,NivelGerencial,TipoResidencia,CondicaoResidencia,ValorImovel,NoAutomoveis,Computador,Lucro
0,2,35.18007,2,1,0,2,3,9,2,2,2,2,1,2,-11.74
1,6,39.80395,2,3,0,3,2,6,3,2,3,3,2,2,-14.96
2,8,54.58819,2,3,0,4,3,2,5,3,3,3,2,2,-20.79
3,10,42.80345,1,3,0,3,3,6,3,2,2,3,1,2,-4.81
4,9,20.77313,1,1,0,1,0,2,1,1,2,1,0,1,71.84


In [ ]:
featNum = ['Idade','NumeroFilhos','Escolaridade','NivelGerencial','TipoResidencia','CondicaoResidencia','ValorImovel','NoAutomoveis','Computador']
featCat = ['Cidade','EstadoCivil','Sexo','Ocupacao','AreaAtuacao']

In [ ]:
x_num = x_train[featNum].reset_index(drop = True)
x_cat = x_train[featCat].reset_index(drop = True)

In [ ]:
x_num = x_num.drop(columns = ['NumeroFilhos','Computador'])

Features Numéricas

In [ ]:
x_num.head(5)

,Idade,Escolaridade,NivelGerencial,TipoResidencia,CondicaoResidencia,ValorImovel,NoAutomoveis
0,35.18007,2,2,2,2,2,1
1,39.80395,3,3,2,3,3,2
2,54.58819,4,5,3,3,3,2
3,42.80345,3,3,2,2,3,1
4,20.77313,1,1,1,2,1,0


Normalização

In [ ]:
x_num_mean = x_num.mean()
x_num_std = x_num.std()

In [ ]:
x_num_norm = (x_num-x_num_mean)/x_num_std

PCA

In [ ]:
correlation = np.corrcoef(x_num,rowvar = False)

In [ ]:
correlation

array([[1.     , 0.75276, 0.86097, 0.71197, 0.73226, 0.76425, 0.72349],
       [0.75276, 1.     , 0.75484, 0.69515, 0.67066, 0.7016 , 0.66667],
       [0.86097, 0.75484, 1.     , 0.74161, 0.79913, 0.83705, 0.80541],
       [0.71197, 0.69515, 0.74161, 1.     , 0.6468 , 0.66913, 0.62404],
       [0.73226, 0.67066, 0.79913, 0.6468 , 1.     , 0.72231, 0.67726],
       [0.76425, 0.7016 , 0.83705, 0.66913, 0.72231, 1.     , 0.71593],
       [0.72349, 0.66667, 0.80541, 0.62404, 0.67726, 0.71593, 1.     ]])

In [ ]:
eigvals,eigvect = np.linalg.eig(correlation)

In [ ]:
eigvals

array([5.37355, 0.10333, 0.4089 , 0.22356, 0.26749, 0.30014, 0.32302])

In [ ]:
eigvect

array([[ 0.39122, -0.35503, -0.02144,  0.81405, -0.18804,  0.14783,  0.02347],
       [ 0.36841,  0.04357, -0.38899, -0.25856,  0.2121 ,  0.72287,  0.27691],
       [ 0.40915,  0.87107,  0.15599,  0.17513, -0.11384, -0.07483, -0.01622],
       [ 0.35693, -0.06554, -0.71179, -0.09916,  0.04845, -0.58826, -0.0585 ],
       [ 0.36933, -0.1437 ,  0.25228, -0.11964,  0.45211,  0.06641, -0.74577],
       [ 0.38142, -0.22404,  0.21369, -0.44844, -0.74293,  0.01942, -0.07237],
       [ 0.36678, -0.19532,  0.456  , -0.12064,  0.38469, -0.3149 ,  0.59806]])

In [ ]:
eigvals/sum(eigvals)

array([0.76765, 0.01476, 0.05841, 0.03194, 0.03821, 0.04288, 0.04615])

In [ ]:
x_avet = eigvect[:,[0,2,5,6]]
x_avet

array([[ 0.39122, -0.02144,  0.14783,  0.02347],
       [ 0.36841, -0.38899,  0.72287,  0.27691],
       [ 0.40915,  0.15599, -0.07483, -0.01622],
       [ 0.35693, -0.71179, -0.58826, -0.0585 ],
       [ 0.36933,  0.25228,  0.06641, -0.74577],
       [ 0.38142,  0.21369,  0.01942, -0.07237],
       [ 0.36678,  0.456  , -0.3149 ,  0.59806]])

In [ ]:
x_num_dec = np.dot(x_num_norm,x_avet)

In [ ]:
x_num_dec = pd.DataFrame(x_num_dec,columns = ['PC0','PC1','PC2','PC3'])

## Features Categoricas

In [ ]:
x_cat

,Cidade,EstadoCivil,Sexo,Ocupacao,AreaAtuacao
0,2,1,2,3,9
1,6,3,2,2,6
2,8,3,2,3,2
3,10,3,1,3,6
4,9,1,1,0,2
...,...,...,...,...,...
1697,4,3,2,4,4
1698,4,3,2,4,4
1699,4,3,2,6,5
1700,5,1,2,3,5


## One Hot Encode pra esses dois

In [ ]:
sexo = pd.get_dummies(x_cat['Sexo'], drop_first=False)
sexo.columns = ['Sexo_masculino', 'Sexo_feminino']

x_cat = pd.concat([x_cat,sexo], axis=1)
del x_cat['Sexo']

In [ ]:
estado_civil = pd.get_dummies(x_cat['EstadoCivil'], drop_first=False)
estado_civil.columns = ['EstadoCivil_solteiro', 'EstadoCivil_junto', 'EstadoCivil_casado', 'EstadoCivil_viuvo']
estado_civil

x_cat = pd.concat([x_cat,estado_civil], axis=1)
del x_cat['EstadoCivil']

In [ ]:
x_cat

,Cidade,Ocupacao,AreaAtuacao,Sexo_masculino,Sexo_feminino,EstadoCivil_solteiro,EstadoCivil_junto,EstadoCivil_casado,EstadoCivil_viuvo
0,2,3,9,0,1,1,0,0,0
1,6,2,6,0,1,0,0,1,0
2,8,3,2,0,1,0,0,1,0
3,10,3,6,1,0,0,0,1,0
4,9,0,2,1,0,1,0,0,0
...,...,...,...,...,...,...,...,...,...
1697,4,4,4,0,1,0,0,1,0
1698,4,4,4,0,1,0,0,1,0
1699,4,6,5,0,1,0,0,1,0
1700,5,3,5,0,1,1,0,0,0


## Feature Hashig para esses três

In [ ]:
from sklearn.feature_extraction import FeatureHasher

In [ ]:
x_cat[['Cidade','Ocupacao','AreaAtuacao']] = x_cat[['Cidade','Ocupacao','AreaAtuacao']].astype('string')

In [ ]:
fh1 = FeatureHasher(n_features=5, input_type='string')
hashed_features = fh1.fit_transform(x_cat['Cidade'])
hashed_features = hashed_features.toarray()
hashCidade = pd.DataFrame(data = hashed_features,columns = ['Cidade0','Cidade1','Cidade2','Cidade3','Cidade4'])
x_cat = pd.concat([x_cat,hashCidade],axis = 1).reset_index(drop = True)
del x_cat['Cidade']

In [ ]:
fh2 = FeatureHasher(n_features=3, input_type='string')
hashed_features = fh2.fit_transform(x_cat['Ocupacao'])
hashed_features = hashed_features.toarray()
hashOcupacao = pd.DataFrame(data = hashed_features,columns = ['Ocupacao0','Ocupacao1','Ocupacao2'])
x_cat = pd.concat([x_cat,hashOcupacao],axis = 1).reset_index(drop = True)
del x_cat['Ocupacao']

In [ ]:
fh3 = FeatureHasher(n_features=3, input_type='string')
hashed_features = fh3.fit_transform(x_cat['AreaAtuacao'])
hashed_features = hashed_features.toarray()
hashAreaAtuacao = pd.DataFrame(data = hashed_features,columns = ['AreaAtuacao0','AreaAtuacao1','AreaAtuacao2'])
x_cat = pd.concat([x_cat,hashAreaAtuacao],axis = 1).reset_index(drop = True)
del x_cat['AreaAtuacao']

In [ ]:
x_cat

,Sexo_masculino,Sexo_feminino,EstadoCivil_solteiro,EstadoCivil_junto,EstadoCivil_casado,EstadoCivil_viuvo,Cidade0,Cidade1,Cidade2,Cidade3,Cidade4,Ocupacao0,Ocupacao1,Ocupacao2,AreaAtuacao0,AreaAtuacao1,AreaAtuacao2
0,0,1,1,0,0,0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0
1,0,1,0,0,1,0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0
2,0,1,0,0,1,0,0.0,0.0,0.0,-1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0
3,1,0,0,0,1,0,0.0,0.0,-1.0,0.0,-1.0,1.0,0.0,0.0,0.0,0.0,1.0
4,1,0,1,0,0,0,0.0,1.0,0.0,0.0,0.0,0.0,-1.0,0.0,1.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1697,0,1,0,0,1,0,0.0,0.0,-1.0,0.0,0.0,0.0,0.0,-1.0,0.0,0.0,-1.0
1698,0,1,0,0,1,0,0.0,0.0,-1.0,0.0,0.0,0.0,0.0,-1.0,0.0,0.0,-1.0
1699,0,1,0,0,1,0,0.0,0.0,-1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0
1700,0,1,1,0,0,0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0


In [ ]:
df_cat = pd.concat([x_cat,y_train],axis = 1)

In [ ]:
x_train_novo = pd.concat([x_num_dec,x_cat],axis =  1)

## Acertando o conjunto de treino

In [ ]:
x_num_test = x_test[featNum].drop(columns = ['Computador','NumeroFilhos'])
x_num_test.NivelGerencial = x_num_test.NivelGerencial -1
x_cat_test = x_test[featCat]

In [ ]:
x_test_norm = (x_num_test-x_num_mean)/x_num_std

In [ ]:
x_test_dec = np.dot(x_test_norm,x_avet)

In [ ]:
x_test_dec = pd.DataFrame(x_test_dec,columns = ['PC0','PC1','PC2','PC3'])

In [ ]:
sexo = pd.get_dummies(x_cat_test['Sexo'], drop_first=False)
sexo.columns = ['Sexo_masculino', 'Sexo_feminino']

x_cat_test = pd.concat([x_cat_test,sexo], axis=1)
del x_cat_test['Sexo']  

In [ ]:
estado_civil = pd.get_dummies(x_cat_test['EstadoCivil'], drop_first=False)
estado_civil.columns = ['EstadoCivil_solteiro', 'EstadoCivil_junto', 'EstadoCivil_casado', 'EstadoCivil_viuvo']
estado_civil

x_cat_test = pd.concat([x_cat_test,estado_civil], axis=1)
del x_cat_test['EstadoCivil']

In [ ]:
x_cat_test[['Cidade','Ocupacao','AreaAtuacao']] = x_cat_test[['Cidade','Ocupacao','AreaAtuacao']].astype('string')

In [ ]:
hashed_features = fh1.transform(x_cat_test['Cidade'])
hashed_features = hashed_features.toarray()
hashCidade = pd.DataFrame(data = hashed_features,columns = ['Cidade0','Cidade1','Cidade2','Cidade3','Cidade4'])
x_cat_test = pd.concat([x_cat_test,hashCidade],axis = 1).reset_index(drop = True)
del x_cat_test['Cidade']

In [ ]:
hashed_features = fh2.transform(x_cat_test['Ocupacao'])
hashed_features = hashed_features.toarray()
hashOcupacao = pd.DataFrame(data = hashed_features,columns = ['Ocupacao0','Ocupacao1','Ocupacao2'])
x_cat_test = pd.concat([x_cat_test,hashOcupacao],axis = 1).reset_index(drop = True)
del x_cat_test['Ocupacao']

In [ ]:
hashed_features = fh3.transform(x_cat_test['AreaAtuacao'])
hashed_features = hashed_features.toarray()
hashAreaAtuacao = pd.DataFrame(data = hashed_features,columns = ['AreaAtuacao0','AreaAtuacao1','AreaAtuacao2'])
x_cat_test = pd.concat([x_cat_test,hashAreaAtuacao],axis = 1).reset_index(drop = True)
del x_cat_test['AreaAtuacao']

In [ ]:
x_test_novo = pd.concat([x_test_dec,x_cat_test],axis =  1)

## Algoritmos de ML

SVM

In [ ]:
from sklearn.svm import SVC
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

In [ ]:
svm = SVC(C=1.0,
          kernel='sigmoid',        #kernel{‘linear’, ‘poly’, ‘rbf’, ‘sigmoid’, ‘precomputed’}, default=’rbf’       
          gamma='auto' ,           # se gamma='scale'(padrão) é passado, então ele usa 1 / (n_features * X.var ()) como valor de gama,
                                   # se 'auto', usa 1 / n_features.
          degree = 3,              # grau do polinomio para kernel poly
          coef0 = 5,               # usa em poly e sigmoid
          tol=0.1,               # Tolerância para critério de parada.
          cache_size=2048,         # Especifique o tamanho do cache do kernel (em MB).
          verbose= False, 
          max_iter=-1,             # Limite rígido nas iterações no solver ou -1 para nenhum limite.
          )

svm.fit(x_train_novo, y_train.values.ravel())

print("Treino\n")
predictions = svm.predict(x_train_novo)
print(confusion_matrix(y_train,predictions))
print(classification_report(y_train,predictions))

print("\n Teste\n")
predictionsTestSvm = svm.predict(x_test_novo)
print(confusion_matrix(y_test,predictionsTestSvm))
print(classification_report(y_test,predictionsTestSvm))

Treino

[[724 127]
 [118 733]]
              precision    recall  f1-score   support

         0.0       0.86      0.85      0.86       851
         1.0       0.85      0.86      0.86       851

    accuracy                           0.86      1702
   macro avg       0.86      0.86      0.86      1702
weighted avg       0.86      0.86      0.86      1702


 Teste

[[178  35]
 [112 813]]
              precision    recall  f1-score   support

         0.0       0.61      0.84      0.71       213
         1.0       0.96      0.88      0.92       925

    accuracy                           0.87      1138
   macro avg       0.79      0.86      0.81      1138
weighted avg       0.89      0.87      0.88      1138



MLP

In [ ]:
from sklearn.neural_network import MLPClassifier

In [ ]:
mlp1 = MLPClassifier(solver = 'sgd',    #adam, sgd, lbfgs
                     activation ='relu',       #identity, logistic, tanh, relu
                     hidden_layer_sizes=(8,8), 
                     alpha = 0.1,
                     learning_rate = 'constant',    #constant, invscaling, adaptive . só para sgd
                     learning_rate_init = 0.005,    #so para sgd
                     power_t = 0.1,                 #so para sgd, usando invscaling
                     momentum = 0.9,                #so para sgd
                     nesterovs_momentum = True,      #so para sgd e se momentum > 0
                     beta_1 = 0.8,                  #so para adam
                     beta_2 = 0.9,                  #so para adam
                     epsilon = 1e-6,                #so para adam
                     tol = 1e-3,
                     max_iter=1000,
                     verbose = False, 
                     early_stopping= True,
                     validation_fraction= 0.2,      #so se early stopping = true
                     shuffle = True, 
                     n_iter_no_change = 50)



mlp1.fit(x_train_novo, 
         y_train.values.ravel())
predictions = mlp1.predict(x_train_novo)
print("Treino\n")
print(confusion_matrix(y_train,predictions))
print(classification_report(y_train,predictions))
print("\n Teste \n")
predictionsTestMlp = mlp1.predict(x_test_novo)
print(confusion_matrix(y_test,predictionsTestMlp))
print(classification_report(y_test,predictionsTestMlp))

Treino

[[685 166]
 [101 750]]
              precision    recall  f1-score   support

         0.0       0.87      0.80      0.84       851
         1.0       0.82      0.88      0.85       851

    accuracy                           0.84      1702
   macro avg       0.85      0.84      0.84      1702
weighted avg       0.85      0.84      0.84      1702


 Teste 

[[173  40]
 [ 89 836]]
              precision    recall  f1-score   support

         0.0       0.66      0.81      0.73       213
         1.0       0.95      0.90      0.93       925

    accuracy                           0.89      1138
   macro avg       0.81      0.86      0.83      1138
weighted avg       0.90      0.89      0.89      1138



## Lucros


Se enviar para todos do "Teste"

In [ ]:
df_test['Lucro'].sum()

20766.38

Se usar os algoritmos:

In [ ]:
df_test.loc[(predictionsTestSvm == 1), 'Lucro'].sum()

22940.86

In [ ]:
df_test.loc[(predictionsTestMlp == 1), 'Lucro'].sum()

23067.61

## Contrapondo Prospect no Modelo para ver quem tem maior acurácia


In [ ]:
df2 = pd.read_excel("/content/gdrive/My Drive/CIÊNCIA DA COMPUTAÇÃO/CNN/Profile_Prospect.xls").drop(columns = ['ClienteNo'])

In [ ]:
df2['Lucro'] = df2['PrimeiraCompra'].copy() - 42

In [ ]:
df2.NivelGerencial = df2.NivelGerencial - 1

In [ ]:
df2.loc[df2.PrimeiraCompra < 42, "PrimeiraCompra"] = 0
df2.loc[df2.PrimeiraCompra >=42, "PrimeiraCompra"] = 1

In [ ]:
x_prof = df2.drop(columns = 'PrimeiraCompra')
y_prof = df2['PrimeiraCompra']

In [ ]:
featNum = ['Idade','NumeroFilhos','Escolaridade','NivelGerencial','TipoResidencia','CondicaoResidencia','ValorImovel','NoAutomoveis','Computador']
featCat = ['Cidade','EstadoCivil','Sexo','Ocupacao','AreaAtuacao']

In [ ]:
x_num_prof = x_prof[featNum].reset_index(drop = True)
x_cat_prof = x_prof[featCat].reset_index(drop = True)

In [ ]:
x_num_prof = x_prof[featNum].drop(columns = ['Computador','NumeroFilhos'])
x_cat_prof = x_prof[featCat]

In [ ]:
x_prof_norm = (x_num_prof-x_num_mean)/x_num_std

In [ ]:
x_prof_dec = np.dot(x_prof_norm,x_avet)

In [ ]:
x_prof_dec = pd.DataFrame(x_prof_dec,columns = ['PC0','PC1','PC2','PC3'])

In [ ]:
sexo = pd.get_dummies(x_cat_prof['Sexo'], drop_first=False)
sexo.columns = ['Sexo_masculino', 'Sexo_feminino']

x_cat_prof = pd.concat([x_cat_prof,sexo], axis=1)
del x_cat_prof['Sexo']  

In [ ]:
estado_civil = pd.get_dummies(x_cat_prof['EstadoCivil'], drop_first=False)
estado_civil.columns = ['EstadoCivil_solteiro', 'EstadoCivil_junto', 'EstadoCivil_casado', 'EstadoCivil_viuvo']
estado_civil

x_cat_prof = pd.concat([x_cat_prof,estado_civil], axis=1)
del x_cat_prof['EstadoCivil']

In [ ]:
x_cat_prof[['Cidade','Ocupacao','AreaAtuacao']] = x_cat_prof[['Cidade','Ocupacao','AreaAtuacao']].astype('string')

In [ ]:
hashed_features = fh1.transform(x_cat_prof['Cidade'])
hashed_features = hashed_features.toarray()
hashCidade = pd.DataFrame(data = hashed_features,columns = ['Cidade0','Cidade1','Cidade2','Cidade3','Cidade4'])
x_cat_prof = pd.concat([x_cat_prof,hashCidade],axis = 1).reset_index(drop = True)
del x_cat_prof['Cidade']

In [ ]:
hashed_features = fh2.transform(x_cat_prof['Ocupacao'])
hashed_features = hashed_features.toarray()
hashOcupacao = pd.DataFrame(data = hashed_features,columns = ['Ocupacao0','Ocupacao1','Ocupacao2'])
x_cat_prof = pd.concat([x_cat_prof,hashOcupacao],axis = 1).reset_index(drop = True)
del x_cat_prof['Ocupacao']

In [ ]:
hashed_features = fh3.transform(x_cat_prof['AreaAtuacao'])
hashed_features = hashed_features.toarray()
hashAreaAtuacao = pd.DataFrame(data = hashed_features,columns = ['AreaAtuacao0','AreaAtuacao1','AreaAtuacao2'])
x_cat_prof = pd.concat([x_cat_prof,hashAreaAtuacao],axis = 1).reset_index(drop = True)
del x_cat_prof['AreaAtuacao']

In [ ]:
x_prof_novo = pd.concat([x_prof_dec,x_cat_prof],axis =  1)

In [ ]:
predictionsProfSVM = svm.predict(x_prof_novo)
print(confusion_matrix(y_prof,predictionsProfSVM))
print(classification_report(y_prof,predictionsProfSVM))

[[ 752  168]
 [ 280 1256]]
              precision    recall  f1-score   support

         0.0       0.73      0.82      0.77       920
         1.0       0.88      0.82      0.85      1536

    accuracy                           0.82      2456
   macro avg       0.81      0.82      0.81      2456
weighted avg       0.82      0.82      0.82      2456



In [ ]:
predictionsProfMLP = mlp1.predict(x_prof_novo)
print(confusion_matrix(y_prof,predictionsProfMLP))
print(classification_report(y_prof,predictionsProfMLP))

[[ 714  206]
 [ 246 1290]]
              precision    recall  f1-score   support

         0.0       0.74      0.78      0.76       920
         1.0       0.86      0.84      0.85      1536

    accuracy                           0.82      2456
   macro avg       0.80      0.81      0.81      2456
weighted avg       0.82      0.82      0.82      2456



Lucros se mandasse pra todos

In [ ]:
df2['Lucro'].sum()

27290.28

Lucro se mandar apenas para aqueles que o algoritmo predisse

In [ ]:
df2.loc[(predictionsProfSVM == 1), 'Lucro'].sum()

36974.47

In [ ]:
df2.loc[(predictionsProfMLP == 1), 'Lucro'].sum()

37128.46